<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# GIS

This notebook shows how to export model data so it can be viewed in GIS (QGIS or ArcMAP).

### Contents<a name="TOC"></a>
1. [Model types](#modeltypes)
2. [Export vector data](#vectordata)
3. [Export raster data](#rasterdata)
4. [Add symbology (QGIS)](#symbology)

In [1]:
import nlmod
import os
import flopy
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

import logging
from IPython.display import FileLink, FileLinks

In [2]:
print(f'nlmod version: {nlmod.__version__}')

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)

nlmod version: 0.2.0b


### [1. Model types](#TOC)<a name="modeltypes"></a>



#### structured grid



In [3]:
model_ws = 'model1'
model_name = 'IJmuiden'

In [4]:
model_ds_struc = xr.load_dataset(os.path.join(model_ws, 'cache', 'full_model_ds.nc'))

In [5]:
model_ds_struc

<xarray.Dataset>
Dimensions:             (time: 1, x: 100, y: 60, layer: 40)
Coordinates:
  * time                (time) datetime64[ns] 2015-01-01
  * x                   (x) float64 9.505e+04 9.515e+04 ... 1.048e+05 1.05e+05
  * y                   (y) float64 5e+05 4.998e+05 ... 4.942e+05 4.94e+05
  * layer               (layer) object 'AAOP' 'NASC' 'ONAWA' ... 'PZWAz4' 'MSz1'
Data variables: (12/17)
    time_steps          (time) int32 0
    idomain             (layer, y, x) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    first_active_layer  (y, x) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    bot                 (layer, y, x) float64 -13.07 -13.07 ... -218.2 -218.5
    top                 (y, x) float64 0.0 0.0 0.0 0.0 ... 2.29 1.49 0.5 0.37
    kh                  (layer, y, x) float64 10.0 10.0 10.0 ... 6.1 6.11 6.11
    ...                  ...
    rws_oppwater_area   (y, x) float64 1e+04 1e+04 1e+04 1e+04 ... 0.0 0.0 0.0
    rws_oppwater_cond   (y, x) float64 1e+03 1e+03 1e+03 1e+03 ... 0.0 0.0 0.0
    rws_oppwater_peil   (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ahn                 (y, x) float64 nan nan nan nan ... 1.971 nan nan 0.2378
    chd                 (layer, y, x) int32 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    recharge            (y, x) float64 0.000606 0.000606 ... 0.0007697 0.0007697
Attributes: (12/24)
    model_name:                IJmuiden
    mfversion:                 mf6
    model_dataset_created_on:  20211206_11:24:11
    exe_name:                  c:\users\oebbe\02_python\nlmod\nlmod\mdims\..\...
    model_ws:                  model1
    figdir:                    model1\figure
    ...                        ...
    delc:                      100.0
    nodata:                    -999
    anisotropy:                10
    fill_value_kh:             1.0
    fill_value_kv:             0.1
    surface_drn_cond:          1000

In [6]:
# create gisdir
gisdir_struc = os.path.join(model_ws, 'gis')
if not os.path.exists(gisdir_struc):
    os.mkdir(gisdir_struc)

#### vertex grid



In [7]:
model_ws = 'model3'
model_name = 'IJm_planeten'

In [8]:
model_ds_vert = xr.load_dataset(os.path.join(model_ws, 'cache', 'full_model_ds.nc'))

# get modelgrid
sim = flopy.mf6.MFSimulation.load(sim_name='mfsim.nam', sim_ws=model_ds_vert.model_ws,
                                  load_only=['DISV'])
gwf = sim.get_model(model_ds_vert.model_name)

# get vertices
model_ds_vert['vertices'] = nlmod.mdims.get_vertices(model_ds_vert, modelgrid=gwf.modelgrid)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    skipping package npf...
    skipping package ic...
    skipping package oc...
    skipping package ghb...
    skipping package drn...
    skipping package chd...
    skipping package rch...
    skipping package ims6...


In [9]:
# create gisdir
gisdir_vert = os.path.join(model_ws, 'gis')
if not os.path.exists(gisdir_vert):
    os.mkdir(gisdir_vert)

In [10]:
model_ds_vert

<xarray.Dataset>
Dimensions:             (time: 6, cid: 6231, layer: 40, vert_per_cid: 4, xy: 2)
Coordinates:
  * time                (time) datetime64[ns] 2005-01-03 ... 2015-01-05
  * cid                 (cid) int32 0 1 2 3 4 5 ... 6226 6227 6228 6229 6230
  * layer               (layer) object 'AAOP' 'NASC' 'ONAWA' ... 'PZWAz4' 'MSz1'
  * vert_per_cid        (vert_per_cid) int32 0 1 2 3
  * xy                  (xy) <U1 'x' 'y'
Data variables: (12/22)
    time_steps          (time) int32 0 1 2 3 4 5
    x                   (cid) float64 9.505e+04 9.515e+04 ... 1.048e+05 1.05e+05
    y                   (cid) float64 5e+05 5e+05 5e+05 ... 4.94e+05 4.94e+05
    idomain             (layer, cid) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    first_active_layer  (cid) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    bot                 (layer, cid) float64 -13.07 -13.07 ... -218.2 -218.5
    ...                  ...
    rws_oppwater_peil   (cid) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ahn                 (cid) float64 nan nan nan nan ... 2.045 nan nan 0.224
    chd                 (layer, cid) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    recharge            (cid, time) float64 0.0006817 -0.0004 ... 0.0004 -0.0004
    rch_name            (cid) object 'rch_1' 'rch_1' 'rch_1' ... 'rch_2' 'rch_2'
    vertices            (cid, vert_per_cid, xy) float64 9.5e+04 ... 4.94e+05
Attributes: (12/25)
    model_name:                IJm_planeten
    mfversion:                 mf6
    model_dataset_created_on:  20211206_11:46:07
    exe_name:                  c:\users\oebbe\02_python\nlmod\nlmod\mdims\..\...
    model_ws:                  model3
    figdir:                    model3\figure
    ...                        ...
    extent:                    [ 95000. 105000. 494000. 500000.]
    nodata:                    -999
    anisotropy:                10
    fill_value_kh:             1.0
    fill_value_kv:             0.1
    surface_drn_cond:          1000

### [2. Export vector data](#TOC)<a name="vectordata"></a>



#### structured

In [11]:
# write model data to a geopackage
fname_geopackage = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_struc, gisdir=gisdir_struc)

# get download link
FileLink(fname_geopackage, result_html_prefix='klik hier om te downloaden -> ')

C:\Users\oebbe\02_python\nlmod\examples\model1\gis\IJmuiden.gpkg

In [12]:
# write model data to multiple shapefiles
fnames = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_struc, driver='ESRI Shapefile', gisdir=gisdir_struc)

# get download link
FileLinks(gisdir_struc, included_suffixes='.shp')

c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:306: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:322: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)


model1\gis/
  ahn.shp
  chd.shp
  idomain.shp
  kh.shp
  kv.shp
  recharge.shp
  rws_oppwater.shp
  rws_oppwater_area.shp
  rws_oppwater_cond.shp
  rws_oppwater_peil.shp
  sea.shp
  starting_head.shp
  topbot.shp

#### vertex grid

In [13]:
model_ds_vert

<xarray.Dataset>
Dimensions:             (time: 6, cid: 6231, layer: 40, vert_per_cid: 4, xy: 2)
Coordinates:
  * time                (time) datetime64[ns] 2005-01-03 ... 2015-01-05
  * cid                 (cid) int32 0 1 2 3 4 5 ... 6226 6227 6228 6229 6230
  * layer               (layer) object 'AAOP' 'NASC' 'ONAWA' ... 'PZWAz4' 'MSz1'
  * vert_per_cid        (vert_per_cid) int32 0 1 2 3
  * xy                  (xy) <U1 'x' 'y'
Data variables: (12/22)
    time_steps          (time) int32 0 1 2 3 4 5
    x                   (cid) float64 9.505e+04 9.515e+04 ... 1.048e+05 1.05e+05
    y                   (cid) float64 5e+05 5e+05 5e+05 ... 4.94e+05 4.94e+05
    idomain             (layer, cid) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    first_active_layer  (cid) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    bot                 (layer, cid) float64 -13.07 -13.07 ... -218.2 -218.5
    ...                  ...
    rws_oppwater_peil   (cid) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ahn                 (cid) float64 nan nan nan nan ... 2.045 nan nan 0.224
    chd                 (layer, cid) int32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    recharge            (cid, time) float64 0.0006817 -0.0004 ... 0.0004 -0.0004
    rch_name            (cid) object 'rch_1' 'rch_1' 'rch_1' ... 'rch_2' 'rch_2'
    vertices            (cid, vert_per_cid, xy) float64 9.5e+04 ... 4.94e+05
Attributes: (12/25)
    model_name:                IJm_planeten
    mfversion:                 mf6
    model_dataset_created_on:  20211206_11:46:07
    exe_name:                  c:\users\oebbe\02_python\nlmod\nlmod\mdims\..\...
    model_ws:                  model3
    figdir:                    model3\figure
    ...                        ...
    extent:                    [ 95000. 105000. 494000. 500000.]
    nodata:                    -999
    anisotropy:                10
    fill_value_kh:             1.0
    fill_value_kv:             0.1
    surface_drn_cond:          1000

In [14]:
# write model data to a geopackage
fname_geopackage = nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_vert, gisdir=gisdir_vert)

In [15]:
# write model data to multiple shapefiles
nlmod.visualise.gis.model_dataset_to_vector_file(model_ds_vert, driver='ESRI Shapefile', gisdir=gisdir_vert)

c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:306: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:322: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)
c:\users\oebbe\02_python\nlmod\nlmod\visualise\gis.py:322: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(fname, driver=driver)


['model3\\gis\\idomain.shp',
 'model3\\gis\\topbot.shp',
 'model3\\gis\\sea.shp',
 'model3\\gis\\starting_head.shp',
 'model3\\gis\\rws_oppwater_cond.shp',
 'model3\\gis\\rws_oppwater_peil.shp',
 'model3\\gis\\rws_oppwater_area.shp',
 'model3\\gis\\recharge.shp',
 'model3\\gis\\ahn.shp',
 'model3\\gis\\chd.shp',
 'model3\\gis\\kv.shp',
 'model3\\gis\\kh.shp']

### [4. Export raster data](#TOC)<a name="rasterdata"></a>

TBA

### [5. Add symbology (QGIS)](#TOC)<a name="symbology"></a>

It is always nice to have automatic symbology for your vector data. Some thoughts:
- QGIS can save symbology of a single shapefile in a .qml file
- In QGIS you can add a .qml file to a geopackage thus saving the symbology to a single file.
- You can create a .qml file in QGIS from existing symbology.
- a .qml file is an xml file so theoretically it is possible to modify a .qml file with Python based on the properties of the data.
- It 



Some limitations of the current gis functions:
- when exporting shapefiles to gis, attributes cannot have names longer
than 10 characters. Now the automatic character shortening of fiona is
used. This is not optimal.
- when exporting data variables with dimension time only the mean values
in time are exported in the shapefile to avoid extremely large files.